In [1]:
from sage.symbolic.expression import Expression
from delierium.MatrixOrder import Context, Mlex, Mgrlex, Mgrevlex
from IPython.display import Math

from delierium.exception import DelieriumNotALinearPDE
from dataclasses import dataclass
from delierium.helpers import latexer
from sage.symbolic.expression_conversions import ExpressionTreeWalker
from IPython.core.debugger import set_trace
import doctest
from typing import Union, Any, Optional, TYPE_CHECKING, cast
from functools import partial, wraps, cache, total_ordering
from operator import mul
from IPython.display import Math

In [2]:
x, y= var("x y")
xi  = function("xi", latex_name=r"\xi")(x,y)
eta = function("eta", latex_name = r"\eta")(x, y)
ctx = Context([xi, eta], [x, y], Mgrevlex)
xi_x = diff(xi, x)
xi_y = diff(xi, y)
xi_xx = diff(xi, x,x)
xi_yy = diff(xi, y,y)
xi_xy = diff(xi, x,y)
eta_x = diff(eta, x)
eta_y = diff(eta, y)
eta_xx = diff(eta, x,x)
eta_yy = diff(eta, y,y)
eta_xy = diff(eta, x,y)

A = function("A")(x,y)
B = function("B")(x,y)
C = function("C")(x,y)
D = function("D")(x,y)
A_x = diff(A, x)
A_y = diff(A, y)
B_x = diff(B, x)
B_y = diff(B, y)
C_x = diff(C, x)
C_y = diff(C, y)
D_x = diff(D, x)
D_y = diff(D, y)

In [3]:
e1 = xi_yy - 2*A*eta_y - B*xi_y + A*xi_x - A_y*eta-A_x*xi
e2 = eta_xx - D*eta_y +C*eta_x +2*D*xi_x +D_y*eta+D_x*xi
e3 = eta_xy - xi_xx/2 +B*eta_x+3*D*xi_y/2+C*xi_x/2 + C_y*eta/2 + C_x*xi/2
e4 = eta_yy - 2*xi_xy +B*eta_y + 3*A*eta_x + 2*C*xi_y + B_y*eta +B_x*xi

In [4]:
#def contextualize(func):
#    @wraps(function)
#    def context_wrapper(context, *args):
#        return partial(func, context)
#    return context_wrapper

In [5]:
_Sage_Expression = sage.symbolic.expression.Expression
_Delierium_Context = Context
def gen_summands(context: _Delierium_Context, expression: _Sage_Expression) -> list[_Sage_Expression]:
    try :
        if expression.operator().__name__ == 'add_vararg':
            # XXX: use expression.iterator
            return expression.operands()
    except AttributeError:
        return [expression]
    return [expression]

In [6]:
#@total_ordering
@dataclass
class DT:
    context: Context
    expression: _Sage_Expression
    coefficient: _Sage_Expression
    derivative: _Sage_Expression
    function: _Sage_Expression
    order: tuple[int]
    comparison_order: tuple[int]
    def __eq__(self, other):
        return self.comparison_order == other.comparison_order
    def __gt__(self, other):
        self.context.gt(self.comparison_order,other.comparison_order)
    def __lt__(self, other):
        return self.comparison_order != other.comparison_order and \
        not self.context.gt(self.comparison_order,other.comparison_order)

In [7]:
def is_derivative(e: Any) -> bool:
    """
    Check if an expression is a pure derivative
    >>> x, y= var("x y")
    >>> xi  = function("xi")(x,y)
    >>> ctx = Context([xi], [x,y])
    >>> xi_x = diff(xi, x)
    >>> xi_y = diff(xi, y)
    >>> xi_xx = diff(xi, x,x)
    >>> xi_yy = diff(xi, y,y)
    >>> xi_xy = diff(xi, x,y)
    >>> A = function("A")(x,y)
    >>> A_x = diff(A, x)
    >>> is_derivative(A)
    False
    >>> is_derivative(1)
    False
    >>> is_derivative(A*xi_x)
    False
    >>> is_derivative(xi)
    False
    >>> is_derivative(xi_xy)
    True
    >>> is_derivative(2*xi_x)
    False
    """
    try:
        return e.operator().__class__.__name__ == "FDerivativeOperator"
    except AttributeError:
        return False

doctest.testmod()

/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: 
Importing Euler_Phi from here is deprecated; please use "from sage.arith.misc import Euler_Phi" instead.
See https://github.com/sagemath/sage/issues/30322 for details.
  return hasattr(f, '__wrapped__')
/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: this class is deprecated, do not use
See https://github.com/sagemath/sage/issues/31545 for details.
  return hasattr(f, '__wrapped__')
/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: 
Importing LaurentSeries from here is deprecated; please use "from sage.rings.laurent_series_ring_element import LaurentSeries" instead.
See https://github.com/sagemath/sage/issues/33602 for details.
  return hasattr(f, '__wrapped__')
/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: 
Importing Moebius from here is deprecated; please use "from sage.arith.misc import Moebiu

TestResults(failed=0, attempted=16)

In [8]:
def order_of_derivative(context: _Delierium_Context, e: Any):
    '''Returns the vector of the orders of a derivative respect to its variables
    >>> x, y, z= var("x y z")
    >>> f1  = function("f1")(x,y, z)
    >>> ctx = Context([f1], [x,y, z])
    >>> order_of_derivative(ctx, diff(f1, x, x, x, x))
    [4, 0, 0]
    >>> order_of_derivative(ctx, f1)    
    [0, 0, 0]
    >>> order_of_derivative(ctx, diff(f1, x, x, x, y, z, z))
    [3, 1, 2]
    >>> ctx = Context([f1], [z, x, y])
    >>> order_of_derivative(ctx, diff(f1, x, x, x, x))
    [4, 0, 0]
    >>> order_of_derivative(ctx, diff(f1, x, x, x, y, z, z))
    [3, 1, 2]
    '''
    res = [0] * len(e.variables())
    if not is_derivative(e):
        return res
    for variable in e.variables():
        # XXX. here seems to be the crucial part: order can depend on:
        # - either the order given by sage by
        # -- sage order
        # -- order given by the order given by function definition
        # - the order given by context
        i = context._independent.index(variable)
        res[i] = e.operator().parameter_set().count(i)
    return res
doctest.testmod()    

TestResults(failed=0, attempted=25)

In [9]:
def _compute_comparison_vector(context, func):
    """
    >>> x, y, z= var("x y z")
    >>> f1  = function("f1")(x, y, z)
    >>> f2  = function("f2")(x, y, z)
    >>> f3  = function("f3")(x, y, z)
    >>> ctx = Context([f1, f2, f3], [x, y, z])
    >>> _compute_comparison_vector(ctx, f1)
    [1, 0, 0]
    >>> _compute_comparison_vector(ctx, f3)
    [0, 0, 1]
    """
    iv = [0] * len(context._dependent)
    if func in context._dependent:
        iv[context._dependent.index(func)] = 1
    elif func.operator() in context._dependent:
        iv[context._dependent.index(func.operator())] = 1
    else:
        pass
    return iv    
    
doctest.testmod()        

TestResults(failed=0, attempted=32)

In [10]:
def compute_order(context, expression):
    """computes the monomial tuple from the derivative part"""
    if is_derivative(expression):
        return order_of_derivative(expression, context, len(context._independent))
    # XXX: Check can that be within a system of linead PDEs ? I think not, as the function must then be identically to zero
    return [0] * len(context._independent)

In [11]:
def _gen_dt(context: _Delierium_Context, expression: _Sage_Expression) -> tuple[tuple[_Sage_Expression], tuple[_Sage_Expression]]:
    coeff = []
    derivs = []
    print (f"{expression=},\n{expression.operator().__class__.__name__=}")
    if str(expression)=="4*x^2*diff(z(x, y), x, x)/y^2 + 2*x*diff(z(x, y), x, y)/y + 2*x^2*diff(z(x, y), y)/y^2 + 2*x^2*diff(w(x, y), x)/y^3 - 2*x*diff(w(x, y), x)/y^2 + x*diff(w(x, y), y)/y^2 - x*w(x, y)/y^3":
        set_trace()

    
    match expression.operator().__class__.__name__:
        case "FDerivativeOperator":
            if expression.operator().function() in context._dependent:
                derivs.append(expression)
            else:
                coeff.append(expression)
        case "Expression":
            print("Symbolic Expression")
        case "NewSymbolicFunction":
            if expression.operator() in context._dependent:
                derivs.append(expression)
            else:
                coeff.append(expression)
        case "function":
            if expression.operator().__name__ == "mul_vararg":
                for operand in expression.operands():
                    if not operand.operator():
                        coeff.append(operand)
                    else:
                        c, d = _gen_dt(context, operand)
                        coeff.extend(c)
                        derivs.extend(d)
        case _:
            coeff.append(expression)
    return coeff, derivs

def gen_dt(context: _Delierium_Context, expression: _Sage_Expression)-> bool:
    expression = expression.expand()
#    if str(expression) == "x*(4*x*diff(z(x, y), x, x)/y + 2*x*diff(z(x, y), y)/y + 2*x*diff(w(x, y), x)/y^2 - 2*diff(w(x, y), x)/y + diff(w(x, y), y)/y - w(x, y)/y^2 + 2*diff(z(x, y), x, y))/y":
#        set_trace()
    c, d = _gen_dt(context, expression)
    if not d:
        raise DelieriumNotALinearPDE(expression)
    d = reduce(mul, d, 1)
    order = order_of_derivative(context, d)
    # XXX here we have our main problem:
    # - if we change the order of the variables with context
    # - then the order of the context and the order of the variables 
    # - as they are used in the functions defined by sagemath differ
    # - so all stuff depending on ordering the variables(and which is
    # - crucial for all our algorithms) gets screwed up. We have to store
    # --- the order as given by context
    # --- the order as given by the canonical sagemath
    # --- a variable which selects which order has to be used in each step    
    return DT(context,
              expression,
              reduce(mul, c, 1), d,
              d.operator().function() if is_derivative(d) else d.function(),              
              order,
              order + _compute_comparison_vector(context, d.operator().function() if is_derivative(d) else d.function())
             )    

In [12]:
gen_dt(ctx, 2*A_x*B_x*xi_xy)
print("1"*20)
gen_dt(ctx, xi)
print("2"*20)
gen_dt(ctx, 2*xi)
print("3"*20)
#gen_dt(ctx, A)
#print("4"*20)
#gen_dt(ctx, 2*A*D)
#print("5"*20)
gen_dt(ctx, xi_x)
print("4"*20)
gen_dt(ctx, A*xi_x)



expression=2*diff(A(x, y), x)*diff(B(x, y), x)*diff(xi(x, y), x, y),
expression.operator().__class__.__name__='function'
expression=diff(A(x, y), x),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=diff(B(x, y), x),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=diff(xi(x, y), x, y),
expression.operator().__class__.__name__='FDerivativeOperator'
11111111111111111111
expression=xi(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
22222222222222222222
expression=2*xi(x, y),
expression.operator().__class__.__name__='function'
expression=xi(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
33333333333333333333
expression=diff(xi(x, y), x),
expression.operator().__class__.__name__='FDerivativeOperator'
44444444444444444444
expression=A(x, y)*diff(xi(x, y), x),
expression.operator().__class__.__name__='function'
expression=A(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
expres

DT(context=<delierium.MatrixOrder.Context object at 0x7f5e13278a00>, expression=A(x, y)*diff(xi(x, y), x), coefficient=A(x, y), derivative=diff(xi(x, y), x), function=xi, order=[1, 0], comparison_order=[1, 0, 1, 0])

In [13]:
gen_dt(ctx, 2*A_x*B_x*xi_xy)

expression=2*diff(A(x, y), x)*diff(B(x, y), x)*diff(xi(x, y), x, y),
expression.operator().__class__.__name__='function'
expression=diff(A(x, y), x),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=diff(B(x, y), x),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=diff(xi(x, y), x, y),
expression.operator().__class__.__name__='FDerivativeOperator'


DT(context=<delierium.MatrixOrder.Context object at 0x7f5e13278a00>, expression=2*diff(A(x, y), x)*diff(B(x, y), x)*diff(xi(x, y), x, y), coefficient=2*diff(A(x, y), x)*diff(B(x, y), x), derivative=diff(xi(x, y), x, y), function=xi, order=[1, 1], comparison_order=[1, 1, 1, 0])

In [14]:
v = list(map(lambda _: gen_dt(ctx, _), gen_summands(ctx, e1)))

expression=-xi(x, y)*diff(A(x, y), x),
expression.operator().__class__.__name__='function'
expression=xi(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
expression=diff(A(x, y), x),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=-eta(x, y)*diff(A(x, y), y),
expression.operator().__class__.__name__='function'
expression=eta(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
expression=diff(A(x, y), y),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=-2*A(x, y)*diff(eta(x, y), y),
expression.operator().__class__.__name__='function'
expression=A(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
expression=diff(eta(x, y), y),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=A(x, y)*diff(xi(x, y), x),
expression.operator().__class__.__name__='function'
expression=A(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
expression=diff(xi(x

In [15]:
def latexer_dt(context, dt):
    def _latex_derivative(context, deriv):
        if is_derivative(deriv):
            func = deriv.function().operator().function()._latex_()
            ps = deriv.operator().parameter_set()
            variables = deriv.operands()
            sub = ",".join(map(lambda _ : variables[_]._latex_(), ps))
            return r"%s_{%s}" % (func, sub)
        else:
            return deriv.function().operator()._latex_()
    def _latex_coeff(context, coeff):
        if str(coeff) == '1':
            return ""
        # need to be more fine granular
        return coeff._latex_()
        
    d = _latex_derivative(context, dt.derivative)
    c = _latex_coeff(context, dt.coefficient)
    return f"{c} {d}"

In [16]:

class DP:
    def __init__(self, context, expression):
        self.context = context
        self.expression = expression.expand()
        print(f"{list(gen_summands(context, self.expression))}")
        self.dts = list(map(lambda _: gen_dt(context, _), gen_summands(context, self.expression)))
        self.dts.sort(reverse = True)
        from collections import Counter        
        counts = Counter(tuple(_.order) for _ in self.dts)
        while duplicates := [_ for _ in counts.items() if _[1] > 1]:
            indices = [i for i, dt in enumerate(self.dts) if tuple(dt.order) == duplicates[0][0]]
            self.dts[indices[0]].coefficient += sum(self.dts[indices[i]].coefficient for i in indices[1:])
            set_trace()
            del self.dts[indices[1:]]
        lcoeff = self.dts[0].coefficient
        for dt in self.dts:
            dt.coefficient /= lcoeff             
        self.lder = self.dts[0].derivative
        self.comp = self.dts[0].comparison_order
        self.lfunc = self.dts[0].function
        self.order = self.dts[0].order
        self.lcoeff = self.dts[0].coefficient
    
    def __lt__ (self, other):
        return self.comp < other.comp


In [17]:
dp=DP(ctx, e1)

[-xi(x, y)*diff(A(x, y), x), -eta(x, y)*diff(A(x, y), y), -2*A(x, y)*diff(eta(x, y), y), A(x, y)*diff(xi(x, y), x), -B(x, y)*diff(xi(x, y), y), diff(xi(x, y), y, y)]
expression=-xi(x, y)*diff(A(x, y), x),
expression.operator().__class__.__name__='function'
expression=xi(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
expression=diff(A(x, y), x),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=-eta(x, y)*diff(A(x, y), y),
expression.operator().__class__.__name__='function'
expression=eta(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
expression=diff(A(x, y), y),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=-2*A(x, y)*diff(eta(x, y), y),
expression.operator().__class__.__name__='function'
expression=A(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
expression=diff(eta(x, y), y),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=A(x, y)*diff(xi(

IndexError: list index out of range

In [ ]:
dp.lder

In [ ]:
for _ in dp.dts:
    print(_.coefficient, _.derivative, _.comparison_order)


In [ ]:
dps=[DP(ctx, _) for _ in [e4,e2,e1,e3]]
for _ in dps:
    print(_.lder)

In [18]:
def _Janet_Basis(dps):
    return sorted(dps)

In [19]:
jb=_Janet_Basis(dps)

NameError: name 'dps' is not defined

In [20]:
def latexer_dp(context , dp):
    return "+".join(map(lambda _ : latexer_dt(context, _), dp.dts)).replace("+-", "-")

In [21]:
latexer_dp(ctx, dps[2])

NameError: name 'dps' is not defined

In [22]:
def _reduce_inner(e1:DP, e2:DP, context:Context) -> DP:
    set_trace()
    for t in (_ for _ in e1.dts if _.function == e2.lfunc):
        # S1 from Algorithm 2.4
        dif = [a - b for a, b in zip(t.order, e2.order)]
        if all(map(lambda h: h == 0, dif)):
            # S2 from Algorithm 2.4
#            print("*****************************")
#            print((e1.expression - e2.expression * t.coefficient).simplify())
#            set_trace()
            return DP(context, e1.expression - e2.expression.mul(t.coefficient))
        if all(map(lambda h: h >= 0, dif)):
            # S2 from Algorithm 2.4
            # toDo: as diff also accepts zeroth derivatives we may
            # unfy these two branches
            #
            # ToDo: take care about the order of variables and oder in context!
            variables_to_diff = []
            # XXX: this may be a problem: which index? that from context or from order?
            for i in range(len(context._independent)):
                if dif[i] != 0:
                    variables_to_diff.extend([context._independent[i]] * abs(dif[i]))
            return DP(context, e1.expression - t.coefficient.mul(diff(e2.expression, *variables_to_diff)))
    return e1

In [23]:
def _reduce(e1:DP, e2:DP, context: Context) -> DP:
    while not bool((_e1 := _reduce_inner(e1, e2, context)) == e1):
        e1 = _e1
    return _e1

In [24]:
def reduceS(e: DP, S: list[DP], context: Context) -> DP:
    reducing = True
    gen = [_ for _ in S]
    while reducing:
        for dp in gen:
            enew = _reduce(e, dp, context)
            # XXX check whether we can replace "==" by 'is'
            if enew == e:
                reducing = False
            else:
                e = enew
                gen = [_ for _ in S]
                reducing = True
    return enew

In [25]:
def autoreduce(context, dps):
    i =  0
    _p, r = dps[:i + 1], dps[i + 1:]
    while r:
        newdps = []
        have_reduced = False
        for _r in r:
            rnew = reduceS(_r, _p, context)
            have_reduced = have_reduced or rnew != _r
            newdps.append(rnew)
        dps = sorted(_p + [_ for _ in newdps if _ not in _p], reverse=True)
        if not have_reduced:
            i += 1
        else:
            i = 0
        _p, r = dps[:i + 1], dps[i + 1:]
    return dps
    
class Janet_Basis:
    def __init__(self, context: Context, expressions: list[_Sage_Expression]):
        self.dps = sorted([DP(context, _) for _ in expressions], reverse = True)
        for _ in self.dps:
            display(Math(latexer_dp(context, _)))
        self.dps = autoreduce(context, self.dps)

In [26]:
vars = var ("x y")
z = function("z")(*vars)
w = function("w")(*vars)
# ctx: items are in descending order
ctx_grevlex_f = Context((w,z), vars, Mgrevlex)
ctx_grlex_f   = Context((w,z), vars, Mgrlex)
ctx_lex_f     = Context((w,z), vars, Mlex)

f1 = diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w/y
f2 = diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w/(2*y**2)
f5 = diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w/(y**2)

system_2_24 = [f1,f2,f3,f4,f5]

In [ ]:
jb = Janet_Basis(ctx_grevlex_f, system_2_24)

[-w(x, y)/y, 1/2*x*diff(z(x, y), y)/((x^2 + y)*y), diff(w(x, y), y)]
expression=-w(x, y)/y,
expression.operator().__class__.__name__='function'
expression=1/y,
expression.operator().__class__.__name__='builtin_function_or_method'
expression=w(x, y),
expression.operator().__class__.__name__='NewSymbolicFunction'
expression=1/2*x*diff(z(x, y), y)/((x^2 + y)*y),
expression.operator().__class__.__name__='function'
expression=1/(x^2 + y),
expression.operator().__class__.__name__='builtin_function_or_method'
expression=1/y,
expression.operator().__class__.__name__='builtin_function_or_method'
expression=diff(z(x, y), y),
expression.operator().__class__.__name__='FDerivativeOperator'
expression=diff(w(x, y), y),
expression.operator().__class__.__name__='FDerivativeOperator'
> /tmp/ipykernel_39175/248931014.py(14)__init__()
     12             self.dts[indices[Integer(0)]].coefficient += sum(self.dts[indices[i]].coefficient for i in indices[Integer(1):])
     13             set_trace()
---> 14

ipdb>  indices[Integer(1):]


[1]


ipdb>  del self.dts[1]


In [ ]:
for _ in jb.dps:
    display(Math(latexer_dp(ctx_grevlex_f, _)))

In [ ]:
# now ist the time to check the sorting....
x, y, z = var("x y z")
u = function ("u")(x,y,z)
ctxMlex = Context((u,),(x,y,z), Mlex)
ctxMgrlex = Context((u,),(x,y,z), Mgrlex)
ctxMgrevlex = Context((u,),(x,y,z), Mgrevlex)
# this is the standard example stolen from wikipedia
u0 = gen_dt(ctxMlex, diff(u,x,3))
u1 = gen_dt(ctxMlex, diff(u,z,2))
u2 = gen_dt(ctxMlex, diff(u,x,y,y,z))
u3 = gen_dt(ctxMlex, diff(u, x,x,z,z))
l1 = [u0, u1,u2,u3]
shuffle(l1)
s = list(sorted(l1, reverse = True))
for _ in s: print(_.order)
#diff(u(x, y, z), z, z)
#diff(u(x, y, z), x, y, y, z)
#diff(u(x, y, z), x, x, z, z)
#diff(u(x, y, z), x, x, x)
#        >>> s = sorted(l1, key=cmp_to_key(lambda item1, item2: item1.sorter(item2)))
#        >>> for _ in s: print(_)
#        diff(u(x, y, z), z, z)
#        diff(u(x, y, z), x, x, x)
#        diff(u(x, y, z), x, y, y, z)
#        diff(u(x, y, z), x, x, z, z)
#        >>> s = sorted(l1, key=cmp_to_key(lambda item1, item2: item1.sorter(item2)))
#        >>> for _ in s: print(_)
#        diff(u(x, y, z), z, z)
#        diff(u(x, y, z), x, x, x)
#        diff(u(x, y, z), x, x, z, z)
#        diff(u(x, y, z), x, y, y, z)
#        '''

In [ ]:
#

In [ ]:
x, y = var('x y')
w = function('w')(x, y)
z = function('z')(x, y)
ctx = Context((w, z), (y, x), Mgrlex)
print(ctx._weight)
l = [gen_dt(ctx, z), gen_dt(ctx, diff(z, x)), gen_dt(ctx, diff(z ,y)), gen_dt(ctx, diff(z ,x,x)), gen_dt(ctx, diff(z ,x, y)), gen_dt(ctx, diff(z ,y, y))] + \
    [gen_dt(ctx, w), gen_dt(ctx, diff(w, x)), gen_dt(ctx, diff(w ,y)), gen_dt(ctx, diff(w ,x,x)), gen_dt(ctx, diff(w ,x, y)), gen_dt(ctx, diff(w ,y, y))]


In [ ]:
shuffle(l)
for _ in sorted(l, reverse=True):
    print(_.expression)

In [ ]:
# https://people.sc.fsu.edu/~jburkardt/py_src/monomial/monomial.html ?